# **API & WEBHOOK-BASED DATA RETRIEVAL**

### **1) REQUIREMENTS SETUP**

In [114]:
# !pip install -r requirements.txt

In [115]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np

### **2) MODULES IMPORT**

In [116]:
from FRED_module import fetch_FRED
from EUROSTAT_module import fetch_EUROSTAT
from WB_module import fetch_WB 
from YFINANCE_module import fetch_YFINANCE
from DBNOMICS_module import fetch_DBNOMICS

### **3) DATA FETCHING**

#### **3.1) EUROSTAT-extracted indicators**

In [180]:
# Industrial Production Indicators by EU member state (monthly, level 1 indicators, Index: 2021=100, non-seasonally adjusted, 1996-01, 2025-08)
# https://ec.europa.eu/eurostat/databrowser/view/sts_inpr_m/default/table?lang=en
# Level 1 Indicators: Mining and Quarrying (B), Manufacturing (C), Electricity, gas, steam and air conditioning supply (D)
EURO_indprod_m_raw = fetch_EUROSTAT(
    "sts_inpr_m",
    filters = {
    "geo": ["EU27_2020", "EU28", "EA20", "EA19",  
    "BE", "BG", "CZ", "DK", "DE", "EE", "IE", "EL", "ES", "FR",
    "HR", "IT", "CY", "LV", "LT", "LU", "HU", "MT", "NL", "AT",
    "PL", "PT", "RO", "SI", "SK", "FI", "SE"],
    "s_adj": "NSA",
    "unit": "I21",
    "nace_r2": ["B", "C", "D"]
    }
)

EURO_indprod_m_raw = EURO_indprod_m_raw[["geo", "nace_r2", "time", "value"]]
EURO_indprod_m_raw = EURO_indprod_m_raw.rename(columns={
    "geo": "Country",
    "time": "Time",
    "nace_r2": "Level 1 Index", 
    "value": "Indprod Index Value (I21)"
})
EURO_indprod_m_raw = EURO_indprod_m_raw.sort_values(["Country", "Level 1 Index", "Time"])

EURO_indprod_m_raw.tail()

,Country,Level 1 Index,Time,Indprod Index Value (I21)
26759,SK,D,2025-04,82.8
26760,SK,D,2025-05,81.0
26761,SK,D,2025-06,79.1
26762,SK,D,2025-07,76.9
26763,SK,D,2025-08,75.1


In [119]:
# Unemployment rate by EU member state (initially quarterly transformed by duplication to monthly, Percentage of population in the labour force (age-class = total, sex-class=total), non-seasonally adjusted, 1994-01 (varying), 2025-08)
# https://ec.europa.eu/eurostat/databrowser/view/une_rt_m/default/table?lang=en
EURO_unem_m_raw = fetch_EUROSTAT(
    "une_rt_m",
    filters = {
    "geo": ["EU27_2020", "EU28", "EA20", "EA19",  
    "BE", "BG", "CZ", "DK", "DE", "EE", "IE", "EL", "ES", "FR",
    "HR", "IT", "CY", "LV", "LT", "LU", "HU", "MT", "NL", "AT",
    "PL", "PT", "RO", "SI", "SK", "FI", "SE"],
    "s_adj": "NSA",
    "unit": "PC_ACT",
    "freq": "M",
    "age": "TOTAL",
    "sex": "T"
    }
)

EURO_unem_m_raw = EURO_unem_m_raw[["geo", "time", "value"]]
EURO_unem_m_raw = EURO_unem_m_raw.rename(columns={
    "geo": "Country",
    "time": "Time",
    "value": "Unemployment Rate (%pop in LF)"
})

EURO_unem_m_raw.tail()

,Country,Time,Unemployment Rate (%pop in LF)
10945,SK,2025-04,5.2
10946,SK,2025-05,5.2
10947,SK,2025-06,5.3
10948,SK,2025-07,5.5
10949,SK,2025-08,5.5


In [120]:
# Gross domestic product at market prices by EU member state (initially quarterly transformed by duplication to monthly, current prices in million euro, non-seasonally adjusted, 1995-Q1 (varying), 2025-Q2)
# https://ec.europa.eu/eurostat/databrowser/view/namq_10_gdp/default/table?lang=en
EURO_GDP_q_raw = fetch_EUROSTAT(
    "namq_10_gdp",
    filters = {
    "geo": ["EU27_2020", "EU28", "EA20", "EA19",  
    "BE", "BG", "CZ", "DK", "DE", "EE", "IE", "EL", "ES", "FR",
    "HR", "IT", "CY", "LV", "LT", "LU", "HU", "MT", "NL", "AT",
    "PL", "PT", "RO", "SI", "SK", "FI", "SE"],
    "na_item": "B1GQ",
    "s_adj": "NSA",
    "unit": "CP_MEUR"
    }
)

EURO_GDP_q_raw = EURO_GDP_q_raw[["geo", "time", "value"]]
EURO_GDP_q_raw = EURO_GDP_q_raw.rename(columns={
    "geo": "Country",
    "time": "Time",
    "value": "GDP (Million EUR)"
})

# Increasing data granularity from quarterly to monthly data by extending the quarter value to single months 
EURO_GDP_q_raw["Time"] = pd.PeriodIndex(EURO_GDP_q_raw["Time"], freq="Q").to_timestamp()
expanded_rows = []

for _, row in EURO_GDP_q_raw.iterrows():
    quarter_end = row["Time"]
    start_month = quarter_end - pd.offsets.QuarterEnd(startingMonth=3) + pd.DateOffset(days=1)
    for i in range(3):
        month = (start_month + pd.DateOffset(months=i)).strftime("%Y-%m")
        expanded_rows.append({
            "Country": row["Country"],
            "Time": month,
            "GDP (Million EUR)": row["GDP (Million EUR)"]
        })

EURO_GDP_m_raw = pd.DataFrame(expanded_rows)
EURO_GDP_m_raw = EURO_GDP_m_raw.sort_values(by=["Country", "Time"]).reset_index(drop=True)

EURO_GDP_m_raw.head()

,Country,Time,GDP (Million EUR)
0,AT,1995-01,43186.6
1,AT,1995-02,43186.6
2,AT,1995-03,43186.6
3,AT,1995-04,46786.8
4,AT,1995-05,46786.8


In [121]:
# HICP by EU member state (monthly, annual rate of change, 1997-01 (varying), 2025-09)
# https://ec.europa.eu/eurostat/databrowser/view/PRC_HICP_MANR__custom_3807536/bookmark/table?lang=en&bookmarkId=cd099aa2-8977-42d5-b5d8-bc5edd3a94df&c=1668007557361 
EURO_HICP_m_raw = fetch_EUROSTAT(
    "prc_hicp_manr",
    filters={
    "geo": [
    "EU27_2020", "EU28", "EA20", "EA19", 
    "BE", "BG", "CZ", "DK", "DE", "EE", "IE", "EL", "ES", "FR",
    "HR", "IT", "CY", "LV", "LT", "LU", "HU", "MT", "NL", "AT",
    "PL", "PT", "RO", "SI", "SK", "FI", "SE"
    ],
    "coicop": "CP00", 
    "unit": "RCH_A"
    }
)

EURO_HICP_m_raw = EURO_HICP_m_raw[["geo", "time", "value"]]
EURO_HICP_m_raw = EURO_HICP_m_raw.rename(columns={
    "geo": "Country",
    "time": "Time",
    "value": "HICP (%, annual rate of change)"
})

EURO_HICP_m_raw.tail()

,Country,Time,"HICP (%, annual rate of change)"
10446,SK,2025-05,4.3
10447,SK,2025-06,4.6
10448,SK,2025-07,4.6
10449,SK,2025-08,4.4
10450,SK,2025-09,4.6


#### **3.2) FRED-extracted indicators**

In [140]:
# US-EUR exchange rate, U.S. Dollars to One Euro (initially daily, converted to monthly, non-seasonally adjusted, 1999-01, 2025-10)
# https://fred.stlouisfed.org/series/DEXUSEU
EXUSEU_d_raw = fetch_FRED("DEXUSEU") 
EXUSEU_d_raw = EXUSEU_d_raw.rename(columns= 
        {"date": "Time", 
         "DEXUSEU": "USD-EUR Spot Exchange Rate"
})

# Aggregation (dimension from daily to monthly)
EXUSEU_d_raw = EXUSEU_d_raw.set_index("Time")
EXUSEU_m_raw = EXUSEU_d_raw.resample("M").mean().reset_index()
EXUSEU_m_raw["Time"] = EXUSEU_m_raw["Time"].dt.to_period("M").astype(str)

# Extension to every Euro-adopting country
euro_countries = [
    "BE", "DE", "EE", "IE", "EL", "ES", "FR",
    "HR", "IT", "CY", "LV", "LT", "LU", "MT",
    "NL", "AT", "PT", "SI", "SK", "FI"
]
EXUSEU_m_raw = (EXUSEU_m_raw.assign(key=1).merge(pd.DataFrame({"Country": euro_countries, "key": 1}), on="key").drop("key", axis=1)).sort_values(["Country", "Time"]).reset_index(drop=True)
EXUSEU_m_raw = EXUSEU_m_raw[["Country", "Time", "USD-EUR Spot Exchange Rate"]]

EXUSEU_m_raw.tail()

,Country,Time,USD-EUR Spot Exchange Rate
6435,SK,2025-06,1.153365
6436,SK,2025-07,1.167082
6437,SK,2025-08,1.164748
6438,SK,2025-09,1.173871
6439,SK,2025-10,1.164894


In [123]:
# US-BGN exchange rate, U.S. Dollars to One Bulgarian Lev (monthly, non-seasonally adjusted, 1960-01, 2021-06)
# https://fred.stlouisfed.org/series/BGRCCUSMA02STM

EXUSBGN_m_raw = fetch_FRED("BGRCCUSMA02STM") 
EXUSBGN_m_raw = EXUSBGN_m_raw.rename(columns= 
        {"date": "Time", 
         "BGRCCUSMA02STM": "USD-BGN Spot Exchange Rate"
})

EXUSBGN_m_raw["Time"] = EXUSBGN_m_raw["Time"].dt.to_period("M").astype(str)

EXUSBGN_m_raw["Country"] = "BG"
EXUSBGN_m_raw = EXUSBGN_m_raw[["Country", "Time", "USD-BGN Spot Exchange Rate"]]

EXUSBGN_m_raw.tail()

,Country,Time,USD-BGN Spot Exchange Rate
769,BG,2021-02,1.6167
770,BG,2021-03,1.6447
771,BG,2021-04,1.6366
772,BG,2021-05,1.6084
773,BG,2021-06,1.6237


In [124]:
# US-SKK exchange rate, U.S. Dollar to One Swedish Kronor (initially SKK-US daily, converted to US-SKK monthly, non-seasonally adjusted, 1971-01, 2025-10)
# https://fred.stlouisfed.org/series/DEXSDUS
EXSKKUS_d_raw = fetch_FRED("DEXSDUS") 
EXSKKUS_d_raw = EXSKKUS_d_raw.rename(columns= 
        {"date": "Time", 
         "DEXSDUS": "SKK-USD Spot Exchange Rate"
})

# Aggregation (dimension from daily to monthly)
EXSKKUS_d_raw = EXSKKUS_d_raw.set_index("Time")
EXSKKUS_m_raw = EXSKKUS_d_raw.resample("M").mean().reset_index()
EXSKKUS_m_raw["Time"] = EXSKKUS_m_raw["Time"].dt.to_period("M").astype(str)

# Conversion to USXS spot exchange rate 
EXUSSKK_m_raw = EXSKKUS_m_raw.copy()
EXUSSKK_m_raw["USD-SKK Spot Exchange Rate"] = 1 / EXUSSKK_m_raw["SKK-USD Spot Exchange Rate"]
EXUSSKK_m_raw = EXUSSKK_m_raw.drop(columns=["SKK-USD Spot Exchange Rate"])

EXUSSKK_m_raw["Country"] = "SE"
EXUSSKK_m_raw = EXUSSKK_m_raw[["Country", "Time", "USD-SKK Spot Exchange Rate"]]

EXUSSKK_m_raw.tail()

,Country,Time,USD-SKK Spot Exchange Rate
653,SE,2025-06,0.104732
654,SE,2025-07,0.104194
655,SE,2025-08,0.104412
656,SE,2025-09,0.106693
657,SE,2025-10,0.106036


In [125]:
# US-DKK exchange rate, U.S. Dollar to One Danish Krone (initially DN-US converted to US-DN, monthly non-seasonally adjusted, 1971-01, 2025-09)
# https://fred.stlouisfed.org/series/EXDNUS
EXDKKUS_m_raw = fetch_FRED("EXDNUS") 
EXDKKUS_m_raw = EXDKKUS_m_raw.rename(columns= 
        {"date": "Time", 
         "EXDNUS": "DKK-USD Spot Exchange Rate"
})

EXDKKUS_m_raw["Time"] = EXDKKUS_m_raw["Time"].dt.to_period("M").astype(str)

# Conversion to USDN spot exchange rate 
EXUSDKK_m_raw = EXDKKUS_m_raw.copy()
EXUSDKK_m_raw["USD-DKK Spot Exchange Rate"] = 1 / EXUSDKK_m_raw["DKK-USD Spot Exchange Rate"]
EXUSDKK_m_raw = EXUSDKK_m_raw.drop(columns=["DKK-USD Spot Exchange Rate"])

EXUSDKK_m_raw["Country"] = "DK"
EXUSDKK_m_raw = EXUSDKK_m_raw[["Country", "Time", "USD-DKK Spot Exchange Rate"]]

EXUSDKK_m_raw.tail()

,Country,Time,USD-DKK Spot Exchange Rate
652,DK,2025-05,0.151110
653,DK,2025-06,0.154607
654,DK,2025-07,0.156387
655,DK,2025-08,0.156057
656,DK,2025-09,0.157257


In [126]:
# US-CZK exchange rate, U.S. Dollar to One Czech koruna (monthly non-seasonally adjusted, 1991-01, 2025-09)
# https://fred.stlouisfed.org/series/CCUSMA02CZM618N 
EXUSCZK_m_raw = fetch_FRED("CCUSMA02CZM618N") 
EXUSCZK_m_raw = EXUSCZK_m_raw.rename(columns= 
        {"date": "Time", 
         "CCUSMA02CZM618N": "USD-CZK Spot Exchange Rate"
})

EXUSCZK_m_raw["Time"] = EXUSCZK_m_raw["Time"].dt.to_period("M").astype(str)
EXUSCZK_m_raw["Country"] = "CZ"
EXUSCZK_m_raw = EXUSCZK_m_raw[["Country", "Time", "USD-CZK Spot Exchange Rate"]]

EXUSCZK_m_raw.tail()

,Country,Time,USD-CZK Spot Exchange Rate
412,CZ,2025-05,22.083318
413,CZ,2025-06,21.539476
414,CZ,2025-07,21.087739
415,CZ,2025-08,21.079476
416,CZ,2025-09,20.752818


In [127]:
# US-HUF exchange rate, U.S. Dollar to One Hungarian Forint (monthly non-seasonally adjusted, 1968-01, 2025-09)
# https://fred.stlouisfed.org/series/CCUSMA02HUM618N
EXUSHUF_m_raw = fetch_FRED("CCUSMA02HUM618N") 
EXUSHUF_m_raw = EXUSHUF_m_raw.rename(columns= 
        {"date": "Time", 
         "CCUSMA02HUM618N": "USD-HUF Spot Exchange Rate"
})

EXUSHUF_m_raw["Time"] = EXUSHUF_m_raw["Time"].dt.to_period("M").astype(str)
EXUSHUF_m_raw["Country"] = "HU"
EXUSHUF_m_raw = EXUSHUF_m_raw[["Country", "Time", "USD-HUF Spot Exchange Rate"]]

EXUSHUF_m_raw.tail()

,Country,Time,USD-HUF Spot Exchange Rate
688,HU,2025-05,357.905909
689,HU,2025-06,349.176667
690,HU,2025-07,341.577391
691,HU,2025-08,340.911905
692,HU,2025-09,333.840909


In [128]:
# US-PLN exchange rate, U.S. Dollar to One Polish Zloty (monthly non-seasonally adjusted, 1957-01, 2025-09)
# https://fred.stlouisfed.org/series/CCUSMA02PLM618N
EXUSPLN_m_raw = fetch_FRED("CCUSMA02PLM618N") 
EXUSPLN_m_raw = EXUSPLN_m_raw.rename(columns= 
        {"date": "Time", 
         "CCUSMA02PLM618N": "USD-PLN Spot Exchange Rate"
})

EXUSPLN_m_raw["Time"] = EXUSPLN_m_raw["Time"].dt.to_period("M").astype(str)
EXUSPLN_m_raw["Country"] = "PL"
EXUSPLN_m_raw = EXUSPLN_m_raw[["Country", "Time", "USD-PLN Spot Exchange Rate"]]

EXUSPLN_m_raw.tail()

,Country,Time,USD-PLN Spot Exchange Rate
820,PL,2025-05,3.769886
821,PL,2025-06,3.703057
822,PL,2025-07,3.640478
823,PL,2025-08,3.666586
824,PL,2025-09,3.629195


In [129]:
# US-RON exchange rate, U.S. Dollar to One Romanian Leu (monthly non-seasonally adjusted, 1960-01, 2023-11)
# https://fred.stlouisfed.org/series/CCUSMA02PLM618N
EXUSRON_m_raw = fetch_FRED("ROUCCUSMA02STM") 
EXUSRON_m_raw = EXUSRON_m_raw.rename(columns= 
        {"date": "Time", 
         "ROUCCUSMA02STM": "USD-RON Spot Exchange Rate"
})

EXUSRON_m_raw["Time"] = EXUSRON_m_raw["Time"].dt.to_period("M").astype(str)
EXUSRON_m_raw["Country"] = "RO"
EXUSRON_m_raw = EXUSRON_m_raw[["Country", "Time", "USD-RON Spot Exchange Rate"]]

EXUSRON_m_raw.tail()

,Country,Time,USD-RON Spot Exchange Rate
798,RO,2023-07,4.4689
799,RO,2023-08,4.5294
800,RO,2023-09,4.6470
801,RO,2023-10,4.7027
802,RO,2023-11,4.6035


In [130]:
# Crude Oil Prices: Brent - Europe (monthly, average price, Dollars per barrel, not seasonally adjusted, 1987-01, 2025-09)
# https://fred.stlouisfed.org/series/MCOILBRENTEU

oilprice_m_raw = fetch_FRED("MCOILBRENTEU") 
oilprice_m_raw = oilprice_m_raw.rename(columns= 
        {"date": "Time", 
         "MCOILBRENTEU": "Crude Oil Price (Brent, Europe)"
})

oilprice_m_raw["Time"] = oilprice_m_raw["Time"].dt.to_period("M").astype(str)

oilprice_m_raw.tail()

,Time,"Crude Oil Price (Brent, Europe)"
456,2025-05,64.45
457,2025-06,71.44
458,2025-07,71.04
459,2025-08,67.87
460,2025-09,67.99


In [131]:
# Nominal Broad U.S. Dollar Index (monthly, index Jan 2006=100, not seasonally adjusted, 2006-01, 2025-09)
# https://fred.stlouisfed.org/series/TWEXBGSMTH

usdi_m_raw = fetch_FRED("TWEXBGSMTH") 
usdi_m_raw = usdi_m_raw.rename(columns= 
        {"date": "Time", 
         "TWEXBGSMTH": "Nominal Broad USD Index"
})

usdi_m_raw["Time"] = usdi_m_raw["Time"].dt.to_period("M").astype(str)

usdi_m_raw.tail()

,Time,Nominal Broad USD Index
626,2025-05,122.6869
627,2025-06,120.9747
628,2025-07,120.5266
629,2025-08,120.9844
630,2025-09,120.4534


In [ ]:
# Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity, Quoted on an Investment Basis (from daily to monthly, percent, not seasonally adjusted, 1962-01, 2025-10)
# https://fred.stlouisfed.org/series/DGS10

ustyield_d_raw = fetch_FRED("DGS10") 
ustyield_d_raw = ustyield_d_raw.rename(columns= 
        {"date": "Time", 
         "DGS10": "Market Yield on 10-Year US Trasury Securities"
})

# Aggregation (dimension from daily to monthly)
ustyield_d_raw = ustyield_d_raw.set_index("Time")
ustyield_m_raw = ustyield_d_raw.resample("M").mean().reset_index()
ustyield_m_raw["Time"] = ustyield_m_raw["Time"].dt.to_period("M").astype(str)

ustyield_m_raw.tail()

,Time,Market Yield on 10-Year US Trasury Securities
761,2025-06,4.383500
762,2025-07,4.391818
763,2025-08,4.264762
764,2025-09,4.120476
765,2025-10,4.062353


In [183]:
# CBOE Volatility Index VIX (from daily to monthly, index, not seasonally adjusted, 1990-01, 2025-10)
# https://fred.stlouisfed.org/series/VIXCLS

VIX_d_raw = fetch_FRED("VIXCLS") 
VIX_d_raw = VIX_d_raw.rename(columns= 
        {"date": "Time", 
         "VIXCLS": "CBOE Volatility Index (VIX)"
})

# Aggregation (dimension from daily to monthly)
VIX_d_raw = VIX_d_raw.set_index("Time")
VIX_m_raw = VIX_d_raw.resample("M").mean().reset_index()
VIX_m_raw["Time"] = VIX_m_raw["Time"].dt.to_period("M").astype(str)

VIX_m_raw.tail()

,Time,CBOE Volatility Index (VIX)
425,2025-06,18.403333
426,2025-07,16.381304
427,2025-08,15.750000
428,2025-09,15.789091
429,2025-10,18.331579


#### **3.3) YAHOO!FINANCE-extracted indicators**

In [173]:
# Monthly price and volume of EU country’s stock index (monthly, price (USD), number of securities traded, 2015-01, 2025-10)
# (!!!) Not available on YFinance for Bulgaria, Croatia, Cyprus, Estonia, Greece, Hungary, Latvia, Lithuania, Luxembourg, Malta, Poland, Romania, Slovakia, Slovenia, Portugal and Sweden
# (!!!) Volume column is dangerous, a lot of 0 values, depending on country, it must be carefully handled 

eu_stock_indices_tickers = {
    "AT": "^ATX",
    "BE": "^BFX",
    "CZ": "^PX",
    "DK": "^OMXC25",
    "FI": "^OMXH25",
    "FR": "^FCHI",
    "DE": "^GDAXI",
    "IE": "^ISEQ",
    "IT": "FTSEMIB.MI",
    "NL": "^AEX",
    "ES": "^IBEX"
}

start = "2015-01-01"
end = "2025-10-25"

list_single_country_dfs = []

# We iterate over each country and respective stock index
# We aggregate data through concatenation based on y axis
for country, ticker in eu_stock_indices_tickers.items():
    df = fetch_YFINANCE(ticker, start, end)

# MultiIndex columns
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [col[0] for col in df.columns]
    df = df.rename(columns={
        "Close": "Closing Price (USD)",
        "YearMonth": "Time"
    })
    df["Country"] = country
    df["Stock Index"] = ticker

    df = df[["Country", "Stock Index", "Time", "Closing Price (USD)", "Volume"]]

    list_single_country_dfs.append(df)

EURO_stock_m_raw = pd.concat(list_single_country_dfs, ignore_index=True)

EURO_stock_m_raw.tail()

,Country,Stock Index,Time,Closing Price (USD),Volume
1354,ES,^IBEX,2025-06,13991.900391,2351850100
1355,ES,^IBEX,2025-07,14397.000000,2531475800
1356,ES,^IBEX,2025-08,14935.799805,1894189600
1357,ES,^IBEX,2025-09,15475.000000,2168514900
1358,ES,^IBEX,2025-10,16000.200195,1904922800


#### **3.5) OTHERS-extracted indicators**

In [194]:
# US Import General by NAICS Codes and EU member country (monthly, general custom value in USD, not seasonally adjusted, 2020-01, 2025-12)
# Manually extracted from the USITC DataWeb (U.S. Department of Commerce): https://dataweb.usitc.gov/trade/search/GenImp/NAIC 
# (!!!) We take the general custom value as the tariffs applies on the overall value, regardless of the quantity traded
# (!!!) NAIC codes: 3334, 3353, 3399, 3312, 3331, 3339, 3351, 3357, 3363, 3398

US_import_raw = pd.read_csv("US_import_USITC_raw.csv")
US_import_raw = US_import_raw.rename(columns={
    "SIC_Code" : "NAIC Code",
    "Desciption": "NAIC Code Description", 
    "YearMonth" : "Time",
    "Value": "Import - General custom value (USD)"
})
US_import_raw["Country"] = US_import_raw["Country"].replace(
    {"Czechia (Czech Republic)": "CZ"}
)
US_import_raw = US_import_raw.sort_values(["Country", "NAIC Code", "Time"]).reset_index(drop=True)

US_import_raw.tail()

,Country,NAIC Code,Description,Time,Import - General custom value (USD)
14719,SK,3399,MISCELLANEOUS MANUFACTURED COMMODITIES,2025-08,0
14720,SK,3399,MISCELLANEOUS MANUFACTURED COMMODITIES,2025-09,0
14721,SK,3399,MISCELLANEOUS MANUFACTURED COMMODITIES,2025-10,0
14722,SK,3399,MISCELLANEOUS MANUFACTURED COMMODITIES,2025-11,0
14723,SK,3399,MISCELLANEOUS MANUFACTURED COMMODITIES,2025-12,0


In [ ]:
# US tariffs ratio on EU-derived imports
# https://policy.trade.ec.europa.eu/news/joint-statement-united-states-european-union-framework-agreement-reciprocal-fair-and-balanced-trade-2025-08-21_en
# https://www.whitehouse.gov/fact-sheets/2025/07/fact-sheet-the-united-states-and-european-union-reach-massive-trade-deal/
# https://www.whitehouse.gov/presidential-actions/2025/09/modifying-the-scope-of-reciprocal-tariffs-and-establishing-procedures-for-implementing-trade-and-security-agreements/

### **4) DATA ASSEMBLING**

#### **4.1) Dependent df**

In [148]:
# Aggregate FX market df
# No merging, but concatenation
# Time frame: 1957/01 (varying) - 2025/10 (varying)
# (!!!) We cannot merge them, neither concatenate them as they have no common ground (neither country of exchnage rate)

# Uploading as single dependent_df
EXUSEU_m_raw.to_csv("aggregate_df/EXUSEU_dependent_df.csv", index=False)
EXUSBGN_m_raw.to_csv("aggregate_df/EXUSBGN_dependent_df.csv", index=False)
EXUSSKK_m_raw.to_csv("aggregate_df/EXUSSKK_dependent_df.csv", index=False)
EXUSDKK_m_raw.to_csv("aggregate_df/EXUSDKK_dependent_df.csv", index=False)
EXUSCZK_m_raw.to_csv("aggregate_df/EXUSCZK_dependent_df.csv", index=False)
EXUSHUF_m_raw.to_csv("aggregate_df/EXUSHUF_dependent_df.csv", index=False)
EXUSPLN_m_raw.to_csv("aggregate_df/EXUSPLN_dependent_df.csv", index=False)
EXUSRON_m_raw.to_csv("aggregate_df/EXUSRON_dependent_df.csv", index=False)

In [174]:
# Aggregate Stock Index df
# No merging, no concat, already cleaned 
# Time frame: 2015/01 - 2025/10 (but it can be extended)
# (!!!) Not available on YFinance for Bulgaria, Croatia, Cyprus, Estonia, Greece, Hungary, Latvia, Lithuania, Luxembourg, Malta, Poland, Romania, Slovakia, Slovenia, Portugal and Sweden
# (!!!) Volume column is dangerous, a lot of 0 values, depending on country, it must be carefully handled 

# Uploading in aggregate_df
EURO_stock_m_raw.to_csv("aggregate_df/EURO_stock_dependent_df.csv", index=False)

In [181]:
# Aggregate Industrial Production Indexes df
# No merging, no concat, already cleaned 
# Time frame: 1996/01 - 2025/08 (but it can be extended)
# (!!!) Level 1 Indicators: Mining and Quarrying (B), Manufacturing (C), Electricity, gas, steam and air conditioning supply (D)

# Uploading in aggregate_df
EURO_indprod_m_raw.to_csv("aggregate_df/EURO_indprod_dependent_df.csv", index=False)

#### **4.2) Regressors df (exposure metrics)**

#### **4.3) Regime-shift transition factor df**

In [195]:
# Aggregate US Import Custom Value df
# No merging, no concat, already cleaned 
# Time frame: 2020/01 - 2025/12 (empty obs)
# (!!!) NAIC codes: 3334, 3353, 3399, 3312, 3331, 3339, 3351, 3357, 3363, 3398

# Uploading in aggregate_df
US_import_raw.to_csv("aggregate_df/US_import_regime_df.csv", index=False)

#### **4.4) Global controls df**

In [184]:
# Aggregate Global Control df
# Merging Index: Time
# Time frame: 2006/01 - 2025/09
# (!!!) To run the model we need no NaN in the df, so we dop them, causing a shrinking of the df time range
global_control_df = (oilprice_m_raw.merge(usdi_m_raw, on="Time", how="outer").merge(VIX_m_raw, on="Time", how="outer").merge(ustyield_m_raw, on="Time", how="outer").sort_values("Time").reset_index(drop=True))
global_control_df = global_control_df.dropna()

# Uploading in aggregate_df
global_control_df.to_csv("aggregate_df/global_control_df.csv", index=False)

global_control_df.tail()

,Time,"Crude Oil Price (Brent, Europe)",Nominal Broad USD Index,CBOE Volatility Index (VIX),Market Yield on 10-Year US Trasury Securities
760,2025-05,64.45,122.6869,20.462273,4.423810
761,2025-06,71.44,120.9747,18.403333,4.383500
762,2025-07,71.04,120.5266,16.381304,4.391818
763,2025-08,67.87,120.9844,15.750000,4.264762
764,2025-09,67.99,120.4534,15.789091,4.120476


#### **4.4) Country-specific controls df**

In [149]:
# Aggregate Country-Specific Control df
# Merging Index: Country, Time
# Time frame: 1997/01 - 2025/06
# (!!!) To run the model we need no NaN in the df, so we dop them, causing a shrinking of the df time range
# (!!!) As GDP data are quarterly, we lose at lot here by dropping NaN even for HICP and Unemployment Rate that were insetad fine
country_control_df = (EURO_GDP_m_raw.merge(EURO_HICP_m_raw, on=["Country", "Time"] , how="outer").merge(EURO_unem_m_raw, on= ["Country", "Time"], how="outer").sort_values(["Country", "Time"]).reset_index(drop=True))
country_control_df = country_control_df.dropna()

# Uploading in aggregate_df
country_control_df.to_csv("aggregate_df/country_specific_control_df.csv", index=False)

country_control_df.tail()

,Country,Time,GDP (Million EUR),"HICP (%, annual rate of change)",Unemployment Rate (%pop in LF)
12714,SK,2025-02,30911.1,4.1,5.4
12715,SK,2025-03,30911.1,4.2,5.3
12716,SK,2025-04,34243.7,3.9,5.2
12717,SK,2025-05,34243.7,4.3,5.2
12718,SK,2025-06,34243.7,4.6,5.3
